In [1]:
from google.colab import drive
drive.mount("mnt")

Mounted at mnt


In [2]:
%cd "mnt/My Drive/Colab Notebooks"

/content/mnt/My Drive/Colab Notebooks


In [3]:
!pip install -q import-ipynb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.1 MB/s eta 0:00:00


In [4]:
!pip install -q liesel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 5.2 MB/s eta 0:00:00


In [5]:
import import_ipynb
import functions_2

In [6]:
from functions_2 import generate_gaussian_data, create_model, test_sampling

In [7]:
import jax
import jax.numpy as jnp
import liesel.model as lsl
import liesel.goose as gs
import liesel.contrib.splines as splines
import tensorflow_probability.substrates.jax as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
import numpy as np

In [8]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
import pickle

In [9]:
from liesel.contrib.splines import equidistant_knots, basis_matrix
import tensorflow_probability.substrates.jax.bijectors as tfb
from liesel.distributions.mvn_degen import MultivariateNormalDegenerate

In [10]:
auth.authenticate_user()

In [11]:
num_sim = 100
n = 500
n_params = 20

In [12]:
######################################################################## MODEL WITH X SAMPLING ####################################################################

In [13]:
%%capture
results_of_model_x_sample = test_sampling(num_sim = num_sim, n = n, M = 3, c_u = 0.25, n_knots = n_params, x_sampling = True, x_naive = False)

liesel.goose.builder - WARNING - No jitter functions provided. The initial values won't be jittered
liesel.goose.engine - INFO - Initializing kernels...
liesel.goose.engine - INFO - Done
liesel.goose.engine - INFO - Starting epoch: FAST_ADAPTATION, 75 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 25 transitions, 25 jitted together
liesel.goose.engine - WARNING - Errors per chain for kernel_00: 1, 0, 0, 0 / 25 transitions
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 50 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 100 transitions, 25 jitted together
liesel.goose.engine - WARNING - Errors per chain for kernel_00: 0, 0, 1, 0 / 100 transitions
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 2

In [14]:
with open('model_x_sample_data_0_25.pickle', 'wb') as handle:
    pickle.dump(results_of_model_x_sample, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Authenticate and construct the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

file_metadata = {'name': 'model_x_sample_data_0_25.pickle'}
media = MediaFileUpload('model_x_sample_data_0_25.pickle', mimetype='application/octet-stream')
file = drive_service.files().create(body=file_metadata, media_body=media,
                                    fields='id').execute()
print(F'File ID: {file.get("id")}')

File ID: 1woPT74Yh7TwPC6qBzZMDzBl9x0Pt7Mn3


In [15]:
########################################################## MODEL TRUE #################################################################

In [16]:
%%capture
results_of_model_true = test_sampling(num_sim = num_sim, n = n, M = 3, c_u = 0.25, n_knots = n_params, x_sampling = False, x_naive = False)

liesel.goose.builder - WARNING - No jitter functions provided. The initial values won't be jittered
liesel.goose.engine - INFO - Initializing kernels...
liesel.goose.engine - INFO - Done
liesel.goose.engine - INFO - Starting epoch: FAST_ADAPTATION, 75 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 25 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 50 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 100 transitions, 25 jitted together
liesel.goose.engine - WARNING - Errors per chain for kernel_00: 0, 1, 0, 0 / 100 transitions
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 200 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.

In [17]:
with open('model_x_true_0_25.pickle', 'wb') as handle:
    pickle.dump(results_of_model_true, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Authenticate and construct the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

file_metadata = {'name': 'model_x_true_0_25.pickle'}
media = MediaFileUpload('model_x_true_0_25.pickle', mimetype='application/octet-stream')
file = drive_service.files().create(body=file_metadata, media_body=media,
                                    fields='id').execute()
print(F'File ID: {file.get("id")}')

File ID: 1u5HYq7aBu1hhHnacMGDidwEfSgglfyBU


In [18]:
############################################## NAIVE MODEL #######################################################

In [12]:
%%capture
results_of_model_naive = test_sampling(num_sim = num_sim, n = n, M = 3, c_u = 0.25, n_knots = n_params, x_sampling = False)

liesel.goose.builder - WARNING - No jitter functions provided. The initial values won't be jittered
liesel.goose.engine - INFO - Initializing kernels...
liesel.goose.engine - INFO - Done
liesel.goose.engine - INFO - Starting epoch: FAST_ADAPTATION, 75 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 25 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 50 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 100 transitions, 25 jitted together
liesel.goose.engine - WARNING - Errors per chain for kernel_00: 0, 1, 0, 0 / 100 transitions
liesel.goose.engine - INFO - Finished epoch
liesel.goose.engine - INFO - Starting epoch: SLOW_ADAPTATION, 200 transitions, 25 jitted together
liesel.goose.engine - INFO - Finished epoch
liesel.goose.

In [13]:
with open('model_x_naive_02_0_25.pickle', 'wb') as handle:
    pickle.dump(results_of_model_naive, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Authenticate and construct the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

file_metadata = {'name': 'model_x_naive_02_0_25.pickle'}
media = MediaFileUpload('model_x_naive_02_0_25.pickle', mimetype='application/octet-stream')
file = drive_service.files().create(body=file_metadata, media_body=media,
                                    fields='id').execute()
print(F'File ID: {file.get("id")}')

File ID: 1El8lc6N5WJe71PMylg-gVMCGJuTxOW5_
